In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

## 이전 작업에서 생성한 endpoint_name을 `endpoint_name` 변수에 입력하시기 바랍니다.

- `endpoint_name`
- `endpoint_config_name`

>`endpoint_config_name`은 삭제시 필요합니다.

In [ ]:
endpoint_name = "YOUR_ENDPOINT"

## 이미지 다운로드 유틸리티 함수

- `downlaod`

In [ ]:
import os
from six.moves import urllib
def download(url, path="/tmp"):
    filename = url.split("/")[-1]
    filepath = os.path.join(path,filename)
    if not os.path.exists(filepath):
        urllib.request.urlretrieve(url,filepath)
    return filepath

In [ ]:
urls = ['https://cdn.pixabay.com/photo/2015/12/19/10/54/bird-1099639_960_720.jpg',\
       'https://cdn.pixabay.com/photo/2016/02/15/13/26/horse-1201143_960_720.jpg',\
       'https://cdn.pixabay.com/photo/2016/02/19/15/46/dog-1210559_960_720.jpg',\
       'https://cdn.pixabay.com/photo/2015/09/29/10/14/truck-truck-963637_960_720.jpg']
file_names = []
for url in urls:
    file_names.append(download(url))
print(file_names)

## 이미지 Display 유틸

- disp_img

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def disp_img(file_name, size=(12, 8), dpi=80):
    img = mpimg.imread(file_name)
    fig = plt.figure()
    fig.set_size_inches(size)
    plt.imshow(img)
    plt.axis('off')

In [ ]:
print(file_names[1])
disp_img(file_names[1])

## Endpoint 응답 유틸

- get_response


In [ ]:
import json
import numpy as np
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker') 
def get_response(file_name):
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)
    result = response['Body'].read()
    result = json.loads(result)
    index = np.argmax(result)
    object_categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    ret = "Result: label - " + object_categories[index] + ", probability - " + str(result[index])
    return ret

In [ ]:
for file in file_names:
    print(file.split("/")[-1].split("-")[0])
    print(get_response(file))